In [77]:
import os

import sqlite3
import pandas as pd
import numpy

In [79]:
# Path to the root directory of datasci
dthon_path = os.path.join(os.path.expanduser('~'), 'Projects/datasci')
dthon_db = os.path.join(dthon_path, "datasci.db")

In [80]:
conn = sqlite3.connect(dthon_db)

# Sample SQL query to find the average prices of the drug 3028 in different regions
# Will likely have to weed out pensioner prices?

sql = """select s.postcode,
            round(avg(t.PatientPrice_Amt),2) as avg_patientprice,
            round(avg(t.WholeSalePrice_Amt),2) as avg_wholesaleprice,
            count(s.postcode) as samplesize-- Sample set of transactions for each suburb
        from transactions t
        join stores s on t.Store_ID = s.Store_ID
        where t.Drug_ID = 3028
        --and s.postcode >= 3000
        --and s.postcode < 3500
        and t.PatientPrice_Amt > 0
        group by s.postcode
        order by avg_patientprice desc
        """

# using pandas because I like the head command
df = pd.read_sql_query(sql, conn)

In [81]:
# Might be interesting to see which stores in each suburb contributed to
# these extremes
df.head()

,postcode,avg_patientprice,avg_wholesaleprice,samplesize
0,3018,29.41,14.25,39
1,7253,26.78,7.37,4
2,3104,26.21,11.21,31
3,2536,24.14,9.08,13
4,2225,22.90,12.71,36


In [82]:
df.tail()

,postcode,avg_patientprice,avg_wholesaleprice,samplesize
526,0812,4.75,1.12,1
527,2020,4.75,3.35,3
528,3143,3.00,2.70,2
529,4066,2.54,2.49,4
530,2144,1.95,4.66,1


In [86]:
# Now doing similar things with panda commands, simplifying SQL
# Same result == good

# In summary, acheived a little, but have had to resort to casting
# floats as floats and sql if I want to do anything like sum or mean below??
sql = """select s.postcode, t.PatientPrice_Amt, cast(t.WholeSalePrice_Amt as float) as WholeSalePrice_Amt
        from transactions t
        join stores s on t.Store_ID = s.Store_ID
        where t.Drug_ID = 3028
        --and s.postcode >= 3000
        --and s.postcode < 3500
        and t.PatientPrice_Amt > 0
        """

df = pd.read_sql_query(sql, conn)

In [87]:
aggregations = {
    'PatientPrice_Amt': {
        'Min Patient Price': 'min',
        'Max Patient Price': 'max'
    },
    'WholeSalePrice_Amt': {
        'Avg Patient Price': 'mean',
        'Max Patient Price': 'max'
    }
}

df.groupby('postcode').agg(aggregations)



PatientPrice_Amt                   WholeSalePrice_Amt  \
         Min Patient Price Max Patient Price  Max Patient Price   
postcode                                                          
0812                  4.75              4.75               1.12   
0820                  5.80               5.8               1.86   
2000                  5.60                16               2.70   
2006                 10.35             10.35               2.70   
2007                  7.00                 7               2.70   
2011                 14.65             14.65               2.70   
2018                  5.60               5.6               1.86   
2020                  1.96               6.2               4.66   
2021                  9.90               9.9               1.86   
2022                  5.60             13.45               4.66   
2027                  6.00                 8               1.86   
2029                  9.75              9.75               1.86   
2031                  5.60               6.1               3.72   
2040                  6.00              9.47               2.70   
2045                  5.60               5.8               1.86   
2046                  5.60                 6               2.70   
2068                  5.60               5.9               1.86   
2076                  5.60               5.9               1.86   
2077                  5.60                 6               8.11   
2084                  6.00             17.85              13.98   
2095                  5.60                 6               2.70   
2099                  9.99              9.99               1.77   
2103                  5.60              6.95               2.70   
2107                  5.60               5.8               1.86   
2112                  2.00                12               5.41   
2113                  5.60               5.9               1.86   
2115                  5.60              14.8               5.41   
2117                  5.60               6.1               2.70   
2118                  5.60             15.45               5.41   
2120                  1.96             14.35               5.41   
...                    ...               ...                ...   
6026                  5.60             17.85               1.86   
6027                  7.15             15.25               3.58   
6056                 19.95             20.95               2.70   
6057                 17.05             17.05               5.41   
6060                  6.10               6.1               5.41   
6076                  5.60              16.7               2.70   
6150                 21.20              21.2               8.11   
6154                 21.20              21.4               5.41   
6157                  5.60              24.4              10.82   
6162                 15.95                18               5.58   
6164                  5.60               5.6               1.86   
6173                  5.90               6.1               5.41   
6210                 13.58             15.98               5.41   
6230                  5.60               5.6               5.41   
6430                  5.90               6.1               5.41   
6450                 15.95             15.95               2.70   
6701                 16.95             16.95               2.70   
7000                  5.60               5.6               3.72   
7011                  1.96              8.26               4.66   
7018                  6.00                 6               6.49   
7140                  5.90               6.1               5.41   
7190                  5.60              13.6               5.41   
7216                  5.80               5.8               3.72   
7248                  6.10               6.1               5.41   
7249                  5.60             10.11               9.32   
7250                  5.60             15.69             